<a href="https://colab.research.google.com/github/kmk4444/Retrieval-augmented-generation/blob/main/Part10_multiquery_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hhh")